In [1]:
# Rongrong's math derivation

import torch
import networkx as nx
import numpy as np

'''
- gamma (edge penalty),
- gamma_c (clique param),
- alpha (learning rate),
- beta (momentum),
- T (number of interations to converge per initialization), and
- eta (explration parameter)
'''

# Graph


######## ER ###########
n = 500
p = 0.5
G = nx.gnp_random_graph(n, p, seed=0)


# ######## d reg ###########
# n = 1000
# d = 10
# G = nx.random_regular_graph(d, n)


# compliment graph:
complement_G = nx.complement(G)

num_components = nx.number_connected_components(G)

print("Number of connected components:", num_components)

if num_components == 1:
    print("Graph has one connected component.")
else:
    print("Graph has more than one connected component.")

### Obtain the A_G matrix

adjacency_matrix = nx.adjacency_matrix(G)
adjacency_matrix_dense = adjacency_matrix.todense()
adjacency_matrix_tensor = torch.tensor(adjacency_matrix_dense, dtype=torch.float32)

### Obtain the A_G_hat matrix

adjacency_matrix_comp = nx.adjacency_matrix(complement_G)
adjacency_matrix_dense_comp = adjacency_matrix_comp.todense()
adjacency_matrix_tensor_comp = torch.tensor(adjacency_matrix_dense_comp, dtype=torch.float32)

### Efficient MIS Checking 2: Directly check if a binarized X is a maximal IS without checking whether it is an IS first...

def MIS_checker_efficient_3(X_torch, adjacency_matrix_tensor, adjacency_matrix_tensor_comp):
  n = X_torch.shape[0]
  # binarized X
  X_torch_binarized = X_torch.bool().float()
  # if for some gradient update, we are still at the boundary, then we have maximal IS
  X_torch_binarized_update = X_torch_binarized - 0.1*(-torch.ones(n) + (n*adjacency_matrix_tensor)@X_torch_binarized)
  # Projection to [0,1]
  X_torch_binarized_update[X_torch_binarized_update>=1] =1
  X_torch_binarized_update[X_torch_binarized_update<=0] =0
  if torch.equal(X_torch_binarized, X_torch_binarized_update):
    # we have a maximal IS:
    MIS = torch.nonzero(X_torch_binarized).squeeze()
    # Exit the function with True
    return True, MIS
  # If not a maximal IS, Exit the function with False
  return False, None

### Here, we set gamma, gamma', and beta.

gamma_c = 1.01
degrees_c = dict(complement_G.degree())
max_degree = max(degrees_c.values())
gamma = 2 + max_degree
beta = 0.8
alpha = 1000
ITERATION_T = 150

H = gamma * adjacency_matrix_tensor - gamma_c * adjacency_matrix_tensor_comp

# Precompute 
one = torch.ones(n, dtype=H.dtype, device=H.device)
b = torch.linalg.solve(H, one)

# Degree-based initialization:

# Calculate degrees of all nodes
degrees = dict(G.degree())

# Find the node with the maximum degree
max_degree_node = max(degrees.values())


d = torch.zeros(n)
for node, degree in G.degree():
    d[node] = 1 - (degree/max_degree_node)

d = d / max(d)
#print(d)

### optimization loop using MGD


n = adjacency_matrix_tensor.shape[0]

input_velocity = torch.zeros(n)

exploration_parameter_eta = 2.0

covariance_matrix = exploration_parameter_eta*torch.eye(len(d))


for init in range(100): # This iterates over initializations and run in parallel when we have GPUs

  if init == 0:
    input_tensor = d
  else:
    torch.manual_seed(init)
    input_tensor = torch.normal(mean=d, std=torch.sqrt(torch.diag(covariance_matrix)))

  for iteration in range(ITERATION_T):

      # Compute the gradient
      input_tensor = (1- 2*alpha) * input_tensor + b

      input_tensor = torch.clamp(input_tensor, 0, 1)

      Checker,MIS = MIS_checker_efficient_3(input_tensor, adjacency_matrix_tensor, adjacency_matrix_tensor_comp)
      if Checker:
        print("initialization: ", init, ";         MIS size = ", len(MIS))
        break

Number of connected components: 1
Graph has one connected component.


In [3]:
# One step approach -- This should be just Theorem 3 in the pCQO paper

import torch
import networkx as nx
import numpy as np

# ----- Build a reproducible ER graph -----
n = 500
p = 0.5
G = nx.gnp_random_graph(n, p, seed=0)
G_comp = nx.complement(G)

A = nx.to_numpy_array(G, dtype=float)
Acomp = nx.to_numpy_array(G_comp, dtype=float)
A_t = torch.tensor(A, dtype=torch.float32)
Acomp_t = torch.tensor(Acomp, dtype=torch.float32)

# ----- Parameters from the user -----
gamma_c = 1.01           # gamma' (clique term)
degrees_c = dict(G_comp.degree())
max_degree = max(degrees_c.values())
gamma = 2 + max_degree   # large edge penalty as specified

# ----- One-step stationary point -----
H = gamma * A - gamma_c * Acomp
one = np.ones(n, dtype=float)
H_pinv = np.linalg.pinv(H)           # Moore–Penrose pseudoinverse
x_star = H_pinv @ one                # stationary point (unconstrained)
x_star_t = torch.from_numpy(x_star).float()

# ----- Define the user's MIS checker -----
def MIS_checker_efficient_3(X_torch, adjacency_matrix_tensor, adjacency_matrix_tensor_comp):
    nloc = X_torch.shape[0]
    X_torch_binarized = X_torch.bool().float()
    X_torch_binarized_update = X_torch_binarized - 0.1*(-torch.ones(nloc) + (nloc*adjacency_matrix_tensor)@X_torch_binarized)
    # Projection to [0,1]
    X_torch_binarized_update[X_torch_binarized_update>=1] = 1
    X_torch_binarized_update[X_torch_binarized_update<=0] = 0
    if torch.equal(X_torch_binarized, X_torch_binarized_update):
        MIS = torch.nonzero(X_torch_binarized).squeeze()
        return True, MIS
    return False, None

# ----- Clamp x* to {0,1} (binary clamp) BEFORE checker -----
# Here we clamp-to-binary by thresholding: >0 -> 1, else 0.
x_clamped_binary = (x_star_t > 0).float()

# Run checker on the clamped vector
ok_clamped, MIS_idx_clamped = MIS_checker_efficient_3(x_clamped_binary, A_t, Acomp_t)

# Summarize
summary = {
    "n": n,
    "p": p,
    "gamma": gamma,
    "gamma_prime": gamma_c,
    "x_star_min": float(x_star_t.min()),
    "x_star_max": float(x_star_t.max()),
    "num_ones_after_binary_clamp": int(x_clamped_binary.sum().item()),
    "checker_on_binary_clamp": bool(ok_clamped),
    "MIS_size_if_true": int(MIS_idx_clamped.numel()) if ok_clamped else 0,
}
summary

{'n': 500,
 'p': 0.5,
 'gamma': 293,
 'gamma_prime': 1.01,
 'x_star_min': -0.0033450305927544832,
 'x_star_max': 0.0025947473477572203,
 'num_ones_after_binary_clamp': 252,
 'checker_on_binary_clamp': False,
 'MIS_size_if_true': 0}

In [4]:
import torch
import networkx as nx
import numpy as np
import pandas as pd
import time
import random

# Fixed method parameters
n = 500
p = 0.5
gamma_c = 1.01
exploration_parameter_eta = 2.0
max_iters = 1500
tol = 1e-8
damping0 = 1e-3
ls_iters = 10
alpha0 = 900   # fixed learning rate
beta = 0.6    # fixed backtracking scale
N_TRIALS = 10  # number of random initializations per graph
instances = 10
k = 5

# Functions
def objective(x, H):
    return -x.sum() + 0.5 * x @ (H @ x)

def gradient(x, H):
    return -torch.ones_like(x) + H @ x

def MIS_checker_efficient_3(X, A, A_bar):
    Xb = X.bool().float()
    Xb_up = Xb - 0.1 * (-torch.ones_like(Xb) + (n * A) @ Xb)
    Xb_up.clamp_(0,1)
    if torch.equal(Xb, Xb_up):
        return True, torch.nonzero(Xb).squeeze()
    return False, None

def partition_adjacency(H, k, seed=0):
    """
    Partition the (0/1) symmetric adjacency A into k disjoint symmetric blocks A_blocks[j],
    summing to A. Returns a list length k of dense float32 tensors.
    """
    torch.manual_seed(seed)
    random.seed(seed)
    n = H.shape[0]
    H_blocks = [torch.zeros_like(H) for _ in range(k)]
    accept_counter = 0

    # Work on upper triangle (i < j), assign each edge to a block
    rows, cols = torch.triu_indices(n, n, offset=1)
    for ii, jj in zip(rows.tolist(), cols.tolist()):
        if H[ii, jj].item() != 0.0:
            b = random.randrange(k)  # round-robin also fine
            H_blocks[b][ii, jj] = H[ii, jj]
            H_blocks[b][jj, ii] = H[ii, jj]  # mirror for symmetry

    # Sanity: sum_j A_blocks[j] == A  (up to dtype)
    # Uncomment for debug:
    assert torch.all((sum(H_blocks) == H))
    return H_blocks

def projected_newton(H_real, H_blocks, k, A, A_bar, x0):
    accept_counter = 0
    x = x0.clone()
    damping_list = [damping0 for _ in range(k)]
    I = torch.eye(n, dtype=H_real.dtype)
    # H_damped = H + damping * I
    history = []
    is_MIS = False
    MIS = None

    for it in range(max_iters):
        # Load a subgraph
        H = H_blocks[it % k]
        damping = damping_list[it % k]
        H_damped = H + damping * I
        g = gradient(x, H)
        g_norm = g.norm().item()
        f0 = objective(x, H).item()
        history.append((it, f0, g_norm))
        if g_norm < tol:
            break

        try:
            p = torch.linalg.solve(H_damped, g)
        except RuntimeError:
            damping *= 10
            H_damped = H + damping * I
            continue

        alpha = alpha0
        accepted = False
        for _ in range(ls_iters):
            x_trial = torch.clamp(x - alpha * p, 0.0, 1.0)
            if objective(x_trial, H).item() < f0:
                x = x_trial
                accepted = True
                accept_counter += 1
                break
            alpha *= beta

        if not accepted:
            damping *= 10

        damping = max(damping * 0.5, 1e-7)
        damping_list[it % k] = damping
        # H_damped = H + damping * I
        

        is_MIS, MIS = MIS_checker_efficient_3(x, A, A_bar)
        if is_MIS:
            break

    return x, history, is_MIS, MIS, accept_counter

# Main batch testing over seeds 0-49
summaries = []
for seed in range(instances):
    print('Running seed = ', seed)
    # Build graph for this seed
    g = nx.gnp_random_graph(n, p, seed=seed)
    A = torch.tensor(nx.adjacency_matrix(g).todense(), dtype=torch.float32)
    A_bar = torch.tensor(nx.adjacency_matrix(nx.complement(g)).todense(), dtype=torch.float32)

    # Build H
    degrees_c = dict(nx.complement(g).degree())
    gamma = 2 + max(degrees_c.values())
    H_real = gamma * A - gamma_c * A_bar

    print('Constructing subgraphs')
    H_blocks = partition_adjacency(H_real, k)
    print(H_blocks)

    # Degree init vector
    degrees = dict(g.degree())
    max_deg = max(degrees.values())
    d = torch.zeros(n)
    for node, deg in degrees.items():
        d[node] = 1 - deg / max_deg
    d_init = d / d.max()

    # Covariance
    cov = exploration_parameter_eta * torch.eye(n)

    # Run multiple trials
    trial_stats = []
    for init in range(N_TRIALS):
        print('Running initialization: ', init)
        if init == 0:
            x0 = d_init.clone()
        else:
            torch.manual_seed((seed << 16) + init)
            x0 = torch.normal(mean=d_init, std=torch.sqrt(torch.diag(cov)))
            x0 = torch.clamp(x0, 0.0, 1.0)

        start = time.time()
        x_opt, history, is_MIS, MIS, accept_counter = projected_newton(H_real, H_blocks, k, A, A_bar, x0)
        elapsed = time.time() - start
        trial_stats.append({
            'iters': len(history),
            'time': elapsed,
            'MIS_size': MIS.numel() if is_MIS else 0,
            'accept_counter': accept_counter
        })

    df = pd.DataFrame(trial_stats)
    summaries.append({
        'seed': seed,
        'max_MIS_size': int(df.MIS_size.max()),
        'mean_iters': df.iters.mean(),
        'mean_time': df.time.mean(),
        'accept_counter': df.accept_counter.mean()
    })

# Aggregate and display results
df_summary = pd.DataFrame(summaries)
print(f"Damped Projected Newton on ER({n},{p}) over seeds 0-{instances-1}, 10 trials each, with {k} subgraphs")
print(df_summary)

Running seed =  0
Constructing subgraphs
[tensor([[  0.0000,   0.0000,   0.0000,  ...,   0.0000,   0.0000,   0.0000],
        [  0.0000,   0.0000,   0.0000,  ...,   0.0000,   0.0000, 293.0000],
        [  0.0000,   0.0000,   0.0000,  ...,  -1.0100,   0.0000,   0.0000],
        ...,
        [  0.0000,   0.0000,  -1.0100,  ...,   0.0000,  -1.0100, 293.0000],
        [  0.0000,   0.0000,   0.0000,  ...,  -1.0100,   0.0000,   0.0000],
        [  0.0000, 293.0000,   0.0000,  ..., 293.0000,   0.0000,   0.0000]]), tensor([[  0.,   0.,   0.,  ...,   0.,   0.,   0.],
        [  0.,   0.,   0.,  ...,   0., 293.,   0.],
        [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
        ...,
        [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
        [  0., 293.,   0.,  ...,   0.,   0., 293.],
        [  0.,   0.,   0.,  ...,   0., 293.,   0.]]), tensor([[  0.,   0.,   0.,  ...,   0., 293., 293.],
        [  0.,   0.,   0.,  ..., 293.,   0.,   0.],
        [  0.,   0.,   0.,  ...,   0., 293.,   0.],
    